In [1]:
import boto3
import botocore
import pandas as pd
from IPython.display import display, Markdown

In [16]:
s3 = boto3.resource('s3')
s3

s3.ServiceResource()

In [18]:
bucket = s3.Bucket(name="noaa-global-hourly-pds")
bucket

s3.Bucket(name='noaa-global-hourly-pds')

## Get NameList

In [43]:
prefix = "1901/"
FilesNotFound = True
FileNameList=[]
for obj in bucket.objects.filter(Prefix=prefix):
    FileNameList.append('{0}{1}/{2}'.format('s3://',bucket.name, obj.key))
    print('{0}/{1}'.format(bucket.name, obj.key))
    FilesNotFound = False
if FilesNotFound:
     print("ALERT", "No file in {0}/{1}".format(bucket, prefix))

FileNameList

noaa-global-hourly-pds/1901/02907099999.csv
noaa-global-hourly-pds/1901/02950099999.csv
noaa-global-hourly-pds/1901/02960099999.csv
noaa-global-hourly-pds/1901/02972099999.csv
noaa-global-hourly-pds/1901/02981099999.csv
noaa-global-hourly-pds/1901/22707099999.csv


['s3://noaa-global-hourly-pds/1901/02907099999.csv',
 's3://noaa-global-hourly-pds/1901/02950099999.csv',
 's3://noaa-global-hourly-pds/1901/02960099999.csv',
 's3://noaa-global-hourly-pds/1901/02972099999.csv',
 's3://noaa-global-hourly-pds/1901/02981099999.csv',
 's3://noaa-global-hourly-pds/1901/22707099999.csv']

In [30]:
#start spark session
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("noaa").getOrCreate()
spark

In [31]:
from pyspark import SparkContext, SparkConf
sc = spark.sparkContext
sc

<SparkContext master=yarn appName=noaa>

In [44]:
FileNameList[1]

's3://noaa-global-hourly-pds/1901/02950099999.csv'

In [45]:
rdd = sc.textFile(FileNameList[1])
#rdd = sc.textFile("s3://noaa-global-hourly-pds/1931/12195099999.csv")

In [46]:
rdd.take(5)

['"STATION","DATE","SOURCE","LATITUDE","LONGITUDE","ELEVATION","NAME","REPORT_TYPE","CALL_SIGN","QUALITY_CONTROL","WND","CIG","VIS","TMP","DEW","SLP","GF1"',
 '"02950099999","1901-01-01T06:00:00","4","60.1166666","21.7","4.5","PARAINEN FAGERHOLM, FI","FM-12","99999","V020","140,1,N,0041,1","99999,9,9,N","000000,1,N,9","-0144,1","+9999,9","10271,1","08,99,1,99,9,99,9,99999,9,99,9,99,9"',
 '"02950099999","1901-01-01T13:00:00","4","60.1166666","21.7","4.5","PARAINEN FAGERHOLM, FI","FM-12","99999","V020","200,1,N,0139,1","99999,9,9,N","000000,1,N,9","-0100,1","+9999,9","10259,1","08,99,1,99,9,99,9,99999,9,99,9,99,9"',
 '"02950099999","1901-01-01T20:00:00","4","60.1166666","21.7","4.5","PARAINEN FAGERHOLM, FI","FM-12","99999","V020","200,1,N,0159,1","99999,9,9,N","000000,1,N,9","-0067,1","+9999,9","10239,1","05,99,1,99,9,99,9,99999,9,99,9,99,9"',
 '"02950099999","1901-01-02T06:00:00","4","60.1166666","21.7","4.5","PARAINEN FAGERHOLM, FI","FM-12","99999","V020","250,1,N,0139,1","99999,9,9,N"

In [54]:
rdd.count()

1096

In [67]:
rdd2 = sc.textFile(FileNameList[2])

In [68]:
rdd2.take(5)

['"STATION","DATE","SOURCE","LATITUDE","LONGITUDE","ELEVATION","NAME","REPORT_TYPE","CALL_SIGN","QUALITY_CONTROL","WND","CIG","VIS","TMP","DEW","SLP","GF1","MW1"',
 '"02960099999","1901-01-01T06:00:00","4","61.183","22.617","51.0","RAIJALA, FI","FM-12","99999","V020","250,1,N,0021,1","99999,9,9,N","000000,1,N,9","-0178,1","+9999,9","10232,1","08,99,1,99,9,99,9,99999,9,99,9,99,9",',
 '"02960099999","1901-01-01T13:00:00","4","61.183","22.617","51.0","RAIJALA, FI","FM-12","99999","V020","200,1,N,0021,1","99999,9,9,N","000000,1,N,9","-0144,1","+9999,9","10226,1","08,99,1,99,9,99,9,99999,9,99,9,99,9","72,1"',
 '"02960099999","1901-01-01T20:00:00","4","61.183","22.617","51.0","RAIJALA, FI","FM-12","99999","V020","200,1,N,0031,1","99999,9,9,N","000000,1,N,9","-0117,1","+9999,9","10210,1","07,99,1,99,9,99,9,99999,9,99,9,99,9","72,1"',
 '"02960099999","1901-01-02T06:00:00","4","61.183","22.617","51.0","RAIJALA, FI","FM-12","99999","V020","180,1,N,0051,1","99999,9,9,N","000000,1,N,9","-0083,1","

In [69]:
rdd2.count()

1096

In [70]:
rdd.union(rdd2).count()

2192

In [80]:
len(FileNameList)
range(1,len(FileNameList))

range(1, 6)

In [82]:
i=1
rdd_all=rdd
for i in range(1,len(FileNameList)):
    print(FileNameList[i])
    rdd_sep=sc.textFile(FileNameList[i])
    rdd_all=rdd_all.union(rdd_sep)
    
rdd_sep
rdd_all.count()
    
    

s3://noaa-global-hourly-pds/1901/02950099999.csv
s3://noaa-global-hourly-pds/1901/02960099999.csv
s3://noaa-global-hourly-pds/1901/02972099999.csv
s3://noaa-global-hourly-pds/1901/02981099999.csv
s3://noaa-global-hourly-pds/1901/22707099999.csv


6571

In [85]:
rdd_all.take(5)

['"STATION","DATE","SOURCE","LATITUDE","LONGITUDE","ELEVATION","NAME","REPORT_TYPE","CALL_SIGN","QUALITY_CONTROL","WND","CIG","VIS","TMP","DEW","SLP","GF1"',
 '"02950099999","1901-01-01T06:00:00","4","60.1166666","21.7","4.5","PARAINEN FAGERHOLM, FI","FM-12","99999","V020","140,1,N,0041,1","99999,9,9,N","000000,1,N,9","-0144,1","+9999,9","10271,1","08,99,1,99,9,99,9,99999,9,99,9,99,9"',
 '"02950099999","1901-01-01T13:00:00","4","60.1166666","21.7","4.5","PARAINEN FAGERHOLM, FI","FM-12","99999","V020","200,1,N,0139,1","99999,9,9,N","000000,1,N,9","-0100,1","+9999,9","10259,1","08,99,1,99,9,99,9,99999,9,99,9,99,9"',
 '"02950099999","1901-01-01T20:00:00","4","60.1166666","21.7","4.5","PARAINEN FAGERHOLM, FI","FM-12","99999","V020","200,1,N,0159,1","99999,9,9,N","000000,1,N,9","-0067,1","+9999,9","10239,1","05,99,1,99,9,99,9,99999,9,99,9,99,9"',
 '"02950099999","1901-01-02T06:00:00","4","60.1166666","21.7","4.5","PARAINEN FAGERHOLM, FI","FM-12","99999","V020","250,1,N,0139,1","99999,9,9,N"

In [89]:
spark

In [94]:
def toCSVLine(data):
    return ','.join(str(d) for d in data)

lines = rdd_all.map(toCSVLine)

lines.saveAsTextFile('s3://buckets/noaa-502/1901.csv')

Py4JJavaError: An error occurred while calling o653.saveAsTextFile.
: java.io.IOException: com.amazon.ws.emr.hadoop.fs.shaded.com.amazonaws.services.s3.model.AmazonS3Exception: Access Denied (Service: Amazon S3; Status Code: 403; Error Code: AccessDenied; Request ID: A00C5E08700D014D; S3 Extended Request ID: iAvEqc1lqhlD78T5SSgRo66IrNx1yGAaQCEUokJF/R6Gbszu0NfGNtv6iUFvdOYrA1Vnv80T1+I=), S3 Extended Request ID: iAvEqc1lqhlD78T5SSgRo66IrNx1yGAaQCEUokJF/R6Gbszu0NfGNtv6iUFvdOYrA1Vnv80T1+I=
	at com.amazon.ws.emr.hadoop.fs.s3n.Jets3tNativeFileSystemStore.list(Jets3tNativeFileSystemStore.java:292)
	at com.amazon.ws.emr.hadoop.fs.s3n.S3NativeFileSystem.getFileStatus(S3NativeFileSystem.java:883)
	at org.apache.hadoop.fs.FileSystem.exists(FileSystem.java:1440)
	at com.amazon.ws.emr.hadoop.fs.EmrFileSystem.exists(EmrFileSystem.java:352)
	at org.apache.hadoop.mapred.FileOutputFormat.checkOutputSpecs(FileOutputFormat.java:130)
	at org.apache.spark.internal.io.HadoopMapRedWriteConfigUtil.assertConf(S

In [95]:
rdd_all.saveAsTextFile(r's3://buckets/noaa-502/1901.csv')

Py4JJavaError: An error occurred while calling o713.saveAsTextFile.
: java.io.IOException: com.amazon.ws.emr.hadoop.fs.shaded.com.amazonaws.services.s3.model.AmazonS3Exception: Access Denied (Service: Amazon S3; Status Code: 403; Error Code: AccessDenied; Request ID: B943D758A8D86CE6; S3 Extended Request ID: nYVF6LwHCw007D3zgzncNKHmWDqWEscMlHtVbljYQkT0Qge7k+GUIE5tKItuMyuq8tIR5jnV2bs=), S3 Extended Request ID: nYVF6LwHCw007D3zgzncNKHmWDqWEscMlHtVbljYQkT0Qge7k+GUIE5tKItuMyuq8tIR5jnV2bs=
	at com.amazon.ws.emr.hadoop.fs.s3n.Jets3tNativeFileSystemStore.list(Jets3tNativeFileSystemStore.java:292)
	at com.amazon.ws.emr.hadoop.fs.s3n.S3NativeFileSystem.getFileStatus(S3NativeFileSystem.java:883)
	at org.apache.hadoop.fs.FileSystem.exists(FileSystem.java:1440)
	at com.amazon.ws.emr.hadoop.fs.EmrFileSystem.exists(EmrFileSystem.java:352)
	at org.apache.hadoop.mapred.FileOutputFormat.checkOutputSpecs(FileOutputFormat.java:130)
	at org.apache.spark.internal.io.HadoopMapRedWriteConfigUtil.assertConf(S

In [93]:
#Export Data
from pyspark.sql import SparkSession
from pyspark import SparkContext
df = sqlContext.createDataFrame(rdd_all)
df.write.csv("s3://buckets/noaa-502/", sep=',', header=True,mode="overwrite")
#rdd_all.write.csv("s3://buckets/noaa-502/",mode="overwrite")

TypeError: Can not infer schema for type: <class 'str'>

**delete header rows**

In [84]:
# 对上述结果筛选掉一些常见的介词，如
# header_words = ['"STATION","DATE","SOURCE","LATITUDE","LONGITUDE","ELEVATION","NAME","REPORT_TYPE","CALL_SIGN","QUALITY_CONTROL","WND","CIG","VIS","TMP","DEW","SLP","GF1","MW1"']
# clean_rdd = rdd_all.filter(lambda w: w not in header_words)
# clean_rdd.count()

6570

In [48]:
df = spark.read\
  .format('csv')\
  .option('header', 'true')\
  .option('inferSchema', 'true')\
  .load(FileNameList[1])

In [50]:
df.show(5)

+----------+-------------------+------+----------+---------+---------+--------------------+-----------+---------+---------------+--------------+-----------+------------+-------+-------+-------+--------------------+
|   STATION|               DATE|SOURCE|  LATITUDE|LONGITUDE|ELEVATION|                NAME|REPORT_TYPE|CALL_SIGN|QUALITY_CONTROL|           WND|        CIG|         VIS|    TMP|    DEW|    SLP|                 GF1|
+----------+-------------------+------+----------+---------+---------+--------------------+-----------+---------+---------------+--------------+-----------+------------+-------+-------+-------+--------------------+
|2950099999|1901-01-01 06:00:00|     4|60.1166666|     21.7|      4.5|PARAINEN FAGERHOL...|      FM-12|    99999|           V020|140,1,N,0041,1|99999,9,9,N|000000,1,N,9|-0144,1|+9999,9|10271,1|08,99,1,99,9,99,9...|
|2950099999|1901-01-01 13:00:00|     4|60.1166666|     21.7|      4.5|PARAINEN FAGERHOL...|      FM-12|    99999|           V020|200,1,N,013

## Merge Data

In [47]:
rdd

s3://noaa-global-hourly-pds/1901/02950099999.csv MapPartitionsRDD[8] at textFile at NativeMethodAccessorImpl.java:0

In [2]:
s3 = boto3.client('s3')
s3_resource = boto3.resource('s3')

In [3]:
# def create_bucket(bucket):
#     import logging

#     try:
#         s3.create_bucket(Bucket=bucket)
#     except botocore.exceptions.ClientError as e:
#         logging.error(e)
#         return 'Bucket ' + bucket + ' could not be created.'
#     return 'Created or already exists ' + bucket + ' bucket.'

In [4]:
#create_bucket('open-data-noaa')

ERROR:root:An error occurred (BucketAlreadyExists) when calling the CreateBucket operation: The requested bucket name is not available. The bucket namespace is shared by all users of the system. Please select a different name and try again.


'Bucket open-data-noaa could not be created.'

In [5]:
def list_buckets(match=''):
    response = s3.list_buckets()
    if match:
        print(f'Existing buckets containing "{match}" string:')
    else:
        print('All existing buckets:')
    for bucket in response['Buckets']:
        if match:
            if match in bucket["Name"]:
                print(f'  {bucket["Name"]}')

In [15]:
s3.list_buckets()

{'ResponseMetadata': {'RequestId': '4C6DF56D8448AF7D',
  'HostId': 'T3vPu5Ivo2fIB6c/BLDjag3t5nFvkRt84sbwzE6uka1RUacCp2YvkemyxPNCJo/gHL6jEKMEilU=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'T3vPu5Ivo2fIB6c/BLDjag3t5nFvkRt84sbwzE6uka1RUacCp2YvkemyxPNCJo/gHL6jEKMEilU=',
   'x-amz-request-id': '4C6DF56D8448AF7D',
   'date': 'Sun, 19 Apr 2020 22:04:00 GMT',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Buckets': [{'Name': '502hw4',
   'CreationDate': datetime.datetime(2020, 3, 27, 1, 29, 29, tzinfo=tzlocal())},
  {'Name': '502lab3m233',
   'CreationDate': datetime.datetime(2020, 1, 30, 16, 38, 49, tzinfo=tzlocal())},
  {'Name': 'aws-logs-270109529444-us-east-1',
   'CreationDate': datetime.datetime(2020, 1, 29, 1, 15, 55, tzinfo=tzlocal())},
  {'Name': 'noaa-502',
   'CreationDate': datetime.datetime(2020, 4, 19, 21, 52, 17, tzinfo=tzlocal())}],
 'Owner': {'DisplayName': 'awslabsc0w518423t157519000

In [14]:
list_buckets(match='99')

Existing buckets containing "99" string:


In [7]:
def list_bucket_contents(bucket, match='', size_mb=0):
    bucket_resource = s3_resource.Bucket(bucket)
    total_size_gb = 0
    total_files = 0
    match_size_gb = 0
    match_files = 0
    for key in bucket_resource.objects.all():
        key_size_mb = key.size/1024/1024
        total_size_gb += key_size_mb
        total_files += 1
        list_check = False
        if not match:
            list_check = True
        elif match in key.key:
            list_check = True
        if list_check and not size_mb:
            match_files += 1
            match_size_gb += key_size_mb
            print(f'{key.key} ({key_size_mb:3.0f}MB)')
        elif list_check and key_size_mb <= size_mb:
            match_files += 1
            match_size_gb += key_size_mb
            print(f'{key.key} ({key_size_mb:3.0f}MB)')

    if match:
        print(f'Matched file size is {match_size_gb/1024:3.1f}GB with {match_files} files')            
    
    print(f'Bucket {bucket} total size is {total_size_gb/1024:3.1f}GB with {total_files} files')

In [ ]:
list_bucket_contents(bucket='noaa-isd-pds', match = 'data')

In [ ]:
list_bucket_contents(bucket='noaa-global-hourly-pds', match = '1950')

In [41]:
rdd = sc.textFile("s3://noaa-global-hourly-pds/1931/12195099999.csv")

In [42]:
rdd.take(5)

['"STATION","DATE","SOURCE","LATITUDE","LONGITUDE","ELEVATION","NAME","REPORT_TYPE","CALL_SIGN","QUALITY_CONTROL","WND","CIG","VIS","TMP","DEW","SLP","AA1","AY1","GF1","IA1","KA1","MD1","MW1","EQD"',
 '"12195099999","1931-01-09T12:00:00","4","54.1333333","22.95","186.0","SUWALKI, PL","FM-12","99999","V020","999,1,C,0000,1","00450,1,C,N","002000,1,N,9","-0050,1","-0056,1","10266,1",,"2,1,99,9","99,99,9,08,1,05,1,00450,1,00,1,00,1",,,,,"Q01+000032APCTENQ02+000002APC3  "',
 '"12195099999","1931-01-10T12:00:00","4","54.1333333","22.95","186.0","SUWALKI, PL","FM-12","99999","V020","200,1,N,0010,1","00450,1,C,N","002000,1,N,9","-0061,1","-0067,1","10298,1",,"2,1,99,9","08,99,1,08,1,05,1,00450,1,00,1,00,1",,,"3,1,002,1,+999,9",,',
 '"12195099999","1931-01-13T12:00:00","4","54.1333333","22.95","186.0","SUWALKI, PL","FM-12","99999","V020","180,1,N,0010,1","00780,1,C,N","004000,1,N,9","-0022,1","-0028,1","10111,1",,"2,1,99,9","08,99,1,08,1,05,1,00800,1,00,1,00,1",,,"8,1,004,1,+999,9",,',
 '"1219